## pg_server_cache 
Looking on the cache of a PG Server


# Configuration
Open a connection to the DB. Use a Connection String stored in a .cfg file

In [7]:
import sqlalchemy
import pandas as pd
import configparser
import matplotlib.pyplot as plt 

# Read from the Config file
config = configparser.ConfigParser() 
config.read_file(open(r'../ipynb.cfg'))

con_str = config.get('con_str', 'PG_AIRBASES') 

engine = sqlalchemy.create_engine(con_str)

# print("Connecting with engine " + str(engine))
try:
    connection = engine.connect()
except (Exception, sqlalchemy.exc.SQLAlchemyError) as error:
    print("Error while connecting to PostgreSQL database:", error)


# Pre-Req
Check whether the extension exists

In [8]:

query = """
SELECT CASE WHEN COUNT(*) > 0 THEN 'true' ELSE 'false' END AS extension_exists
FROM pg_extension
WHERE extname = 'pg_stat_statements';
"""
df = pd.read_sql_query(query, connection)
df

,extension_exists
0,true


## BUFFER CACHE
Postgres caches recent queries in memory called the shared buffer cache (shared_buffers in postgresql.conf). The pg_statio_user_tables has as rows representing various stats on each of the (user) tables. The two columns of interest we’ll be looking at are:

* pg_statio_user_tables.heap_blks_read — Number of disk blocks read from a table (ie. missed cache)
* pg_statio_user_tables.heap_blks_hits — Number of buffer hits from this table (ie. cache hit)


In [9]:
query_ccnnections_metrics = """
SELECT 
  schemaname, 
  pg_class.relname, 
  ROUND(
    CASE 
      WHEN heap_blks_hit + heap_blks_read = 0 
      THEN 0 
      ELSE heap_blks_hit::decimal / (heap_blks_hit + heap_blks_read) 
    END, 
    3
  ) as cache_hit_ratio
FROM 
  pg_statio_user_tables 
  JOIN pg_class ON pg_statio_user_tables.relid = pg_class.oid 
  JOIN pg_namespace ON pg_namespace.oid = pg_class.relnamespace;
"""

df = pd.read_sql_query(query_ccnnections_metrics, connection)
df

,schemaname,relname,cache_hit_ratio
0,postgres_air,booking_leg,0.328
1,postgres_air,airport,0.953
2,postgres_air,account,1.000
3,postgres_air,frequent_flyer,1.000
4,postgres_air,booking,0.394
5,postgres_air,passenger,0.829
6,postgres_air,phone,0.642
7,postgres_air,boarding_pass,0.058
8,postgres_air,flight,0.996
9,postgres_air,aircraft,0.875


# pg_stat_database
https://www.postgresql.org/docs/current/monitoring-stats.html#MONITORING-PG-STAT-DATABASE-VIEW


The ```pg_stat_database``` view will contain one row for each database in the cluster, plus one for shared objects, showing database-wide statistics.

In [10]:
query_pg_stat_database = """
select * 
from pg_stat_database 
order by datname
limit 100
"""

df = pd.read_sql_query(query_pg_stat_database, connection)
df



,datid,datname,numbackends,xact_commit,xact_rollback,blks_read,blks_hit,tup_returned,tup_fetched,tup_inserted,...,tup_deleted,conflicts,temp_files,temp_bytes,deadlocks,checksum_failures,checksum_last_failure,blk_read_time,blk_write_time,stats_reset
0,71456,airbases,2,1189650,2328,1311815907,1769191786,253855587519,283977054,107662840,...,1871,0,77437,136098456316,0,0,None,5.106201e+07,16636.023,2022-07-24 13:22:40.397635+00:00
1,16416,ariksa,1,2522565,4291,12232908,118734759,2674756447,25281991,36471769,...,1751,0,182,5273378816,0,0,None,2.095235e+05,6622.894,2022-03-24 18:14:50.881796+00:00
2,741903,bee_hero,1,38142,190,11132,44426694,456806817,6583726,136694,...,43,0,34,130007040,0,0,None,1.496600e+01,0.000,2023-04-13 11:50:31.632107+00:00
3,742423,bee_hero_2,1,27338,30,321,1125979,16342093,192062,110,...,0,0,0,0,0,0,None,7.221000e+00,0.000,2023-04-13 12:10:39.320840+00:00
4,27603,books,0,1738026,144,2030,72158997,1054481058,12203880,33,...,18,0,0,0,0,0,None,5.584080e+02,0.000,2022-05-24 10:05:01.930212+00:00
5,669717,demo,1,747877,46,770349,72057509,574462549,13276673,21810519,...,10,0,569,1494502662,0,0,None,3.107471e+04,1706.707,2023-01-13 06:32:27.650539+00:00
6,620733,platform-v2,9,1827689,19503,380388,1567330043,7256393598,2858484817,1539700,...,139234,0,66951,145336169037,0,0,None,1.823527e+04,296.093,2023-01-29 22:13:50.640788+00:00
7,14301,postgres,3,14657439,388,3270,120165169,1739357721,20409683,1414,...,56,0,658978,3236630531368,0,0,None,1.186259e+03,0.000,2021-09-14 02:26:35.621660+00:00
8,704871,prisma_migrate_shadow_db_15f7c6f6-3839-4e8d-9b...,1,314685,0,645,13086797,227320970,2217479,1301,...,102,0,0,0,0,0,None,1.369100e+01,0.000,2023-03-19 10:44:50.857163+00:00
9,680488,prisma_migrate_shadow_db_1c09cbde-3031-41eb-92...,0,506085,2,630,21027956,355977011,3556326,1095,...,88,0,0,0,0,0,None,1.000300e+01,0.000,2023-03-02 18:20:55.273696+00:00
